# **Работа № 5**. О свойствах различных оптимизаторов:
1. Смастерить прогнозирующую модель (например из Keras, по-простому, без затей)
2. На имеющихся рядах испытать, задавая разные loss-функции и методы оптимизации
3. Сделать выводы о влиянии функции потерь и оптимизаторов на качество прогноза

# Импорт библиотек

In [49]:
pip install yfinance

In [50]:
import yfinance as yf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Загрузка данных

In [51]:
# Берем волатильный актив (биткоин с тикером BTC) с ценами открытия, мин, макс и закрытия

symbols = ['BTC-USD']
data = pd.DataFrame(yf.download(symbols, '2022-10-01', '2022-11-17', interval='1h'))
data = data[['Open', 'High', 'Low', 'Close']]
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close
Datetime,,,,
2022-10-01 00:00:00+00:00,19431.031250,19471.154297,19386.865234,19400.000000
2022-10-01 01:00:00+00:00,19400.132812,19425.039062,19389.675781,19425.039062
2022-10-01 02:00:00+00:00,19424.039062,19426.058594,19404.068359,19422.853516
2022-10-01 03:00:00+00:00,19421.574219,19428.255859,19388.191406,19412.548828
2022-10-01 04:00:00+00:00,19413.632812,19415.117188,19389.947266,19404.400391
...,...,...,...,...
2022-11-16 19:00:00+00:00,16554.515625,16554.962891,16488.027344,16554.962891
2022-11-16 20:00:00+00:00,16555.960938,16581.687500,16534.779297,16580.814453
2022-11-16 21:00:00+00:00,16578.466797,16590.193359,16542.443359,16551.392578


# Построение модели

In [58]:
# Функция для создания модели

def create_model(input_data):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_data,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    return model

In [59]:
# Обучение модели

input_data = data.shape[1]-1
X = data.drop(['Close'], axis=1)
y = data['Close'].values

In [65]:
# Тестирование разных оптимизизаторов и функций потерь

loss_functions = ['mean_squared_error', 'mean_absolute_error', 'huber_loss', 'logcosh']
optimizers = ['adam', 'adagrad', 'sgd', 'rmsprop']

res = pd.DataFrame(columns=['Loss function', 'Optimizer', 'Loss'])

for loss_function in loss_functions:
    for optimizer in optimizers:
        model = create_model(input_data)
        model.compile(optimizer=optimizer, loss=loss_function)
        model.fit(X, y, epochs=50, verbose=0)
        loss = model.evaluate(X, y)
        res.loc[len(res.index)] = [loss_function, optimizer, loss]
        print(f"loss function: {loss_function}, optimizer: {optimizer}")

36/36 [==============================] - 0s 2ms/step - loss: 5502.5684
loss function: mean_squared_error, optimizer: adam
36/36 [==============================] - 0s 2ms/step - loss: 4506780.5000
loss function: mean_squared_error, optimizer: adagrad
36/36 [==============================] - 0s 2ms/step - loss: nan
loss function: mean_squared_error, optimizer: sgd
36/36 [==============================] - 0s 2ms/step - loss: 44760.8281
loss function: mean_squared_error, optimizer: rmsprop
36/36 [==============================] - 0s 2ms/step - loss: 47.6135
loss function: mean_absolute_error, optimizer: adam
36/36 [==============================] - 0s 2ms/step - loss: 43.8586
loss function: mean_absolute_error, optimizer: adagrad
36/36 [==============================] - 0s 3ms/step - loss: 19309.3711
loss function: mean_absolute_error, optimizer: sgd
36/36 [==============================] - 0s 2ms/step - loss: 149.6391
loss function: mean_absolute_error, optimizer: rmsprop
36/36 [=========

In [66]:
# Вывод модели для разных оптимизаторов и функций потерь

pd.options.display.float_format ='{:,.4f}'.format
res.sort_values(by='Loss')

,Loss function,Optimizer,Loss
13,logcosh,adagrad,40.0847
9,huber_loss,adagrad,42.3614
8,huber_loss,adam,42.6811
5,mean_absolute_error,adagrad,43.8586
4,mean_absolute_error,adam,47.6135
12,logcosh,adam,52.9392
7,mean_absolute_error,rmsprop,149.6391
15,logcosh,rmsprop,236.9037
11,huber_loss,rmsprop,265.5132
0,mean_squared_error,adam,"5,502.5684"
